Ev Fiyatlarını Tahmin Etme: Regresyon/Sayısal Tahmin


In [1]:
from keras.datasets import boston_housing

C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
(train_data, train_labels), (test_data, test_labels) =boston_housing.load_data()

57344/57026 [==============================] - 0s 3us/step


In [3]:
train_data.shape

(404, 13)

In [4]:
test_data.shape

(102, 13)

In [7]:
train_labels[45]

15.6

In [8]:
train_data.shape[1]

13

In [17]:
mean = train_data.mean(axis = 0) # train_data için ort hesapla
train_data-=mean  # her değerden mean ı çıkar 
std = train_data.std(axis=0) #train_data için std hesapla
train_data/=std # tüm değerleri std ye böl 

test_data -=mean # test_data yo her değerden çıkar
test_data /= std # test_data yı std ye böl

In [29]:
from keras import layers
from keras import models
# bir fonksiyon ile model oluşturma...
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation = 'relu' , input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation = 'relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer = "rmsprop" ,loss='mse' , metrics=['mae'])
    
    return model

#Son katmanda aktivasyon fonksiyonu kullanmıyoruz çünkü 1-0 gibi bir sınıf tahmini değil,
#sürekli bir sayısal değer tahmini yapıyoruz

#“mae”: sayısal tahmin hata ortalaması

Veri seti küçük olduğu için train/validation/test bölümlemesi yerine, k-fold cross validation kullanacağız. k= 3,5,10 yaygın


In [30]:
# K-FOLD cross validation bloğu...
import numpy as np
k= 4
num_val_samples = len(train_data) //k
num_epochs = 100
all_scores =[]
for i in range(k):
    print('processing fold # ',i)
    #k bölümden birini validation data kalanını train olarak seç
    val_data = train_data[i * num_val_samples:(i+1)* num_val_samples]
    val_labels = train_labels[i * num_val_samples:(i+1)* num_val_samples]
    
    # yukarıda yapılan seçim dışında kalan k bölümleri train data olarak kullan
    partial_train_data = np.concatenate(
        [train_data[:i*num_val_samples],
        train_data[(i+1)*num_val_samples:]],
        axis =0 )
    
    partial_train_labels = np.concatenate(
        [train_labels[:i*num_val_samples],
        train_labels[(i+1)*num_val_samples:]],
        axis =0 )
    
    # yukarıda compile edilen modeli build et
    model = build_model()
    
    #train/fit bölümü verbose 0 ise detaylar ekrana gelmeyecek 'gösterme 'demek
    model.fit(partial_train_data,partial_train_labels,
              epochs = num_epochs ,batch_size =1,verbose =0)
    # fit edilen eğitilen bölümü validation bölümü ile test et 
    val_mse,val_mae = model.evaluate(val_data,val_labels,verbose=0)
    all_scores.append(val_mae)
    
    


processing fold #  0
processing fold #  1
processing fold #  2
processing fold #  3


In [31]:
all_scores 

[2.2780409520215326, 2.227260986177048, 2.9048202887620076, 2.3188645774775214]

In [33]:
np.mean(all_scores) #tahminda ort 2400 dolar lık hata var

2.4322467011095275